In [ ]:
import socket
import time
import cv2
import numpy

import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
def Init(s):
    s.bind(address)

def Connect(s):
    s.listen(1)
    conn, addr = s.accept()
    print('connect from:'+str(addr))

    return conn

def recvall(sock, count):
    buf = b'' # buf是一个byte类型
    while count:
        # 接受TCP套接字的数据，数据以字符串形式返回，count指定要接收的最大数据量
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf


def ReceiveVideo(conn):
    
    start = time.time() # 用于计算帧率信息
    length = recvall(conn,16) # 获得图片文件的长度,16代表获取长度
    stringData = recvall(conn, int(length)) # 根据获得的文件长度，获取图片文件
    data = numpy.frombuffer(stringData, numpy.uint8) # 将获取到的字符流数据转换成1维数组
    decimg=cv2.imdecode(data,cv2.IMREAD_COLOR) # 将数组解码成图像

    if Detection_Flag:
        with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
            #image = cv2.flip(cv2.imread("./test.jpg"), 1)
            image=decimg
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            print('Handedness:', results.multi_handedness)
            if results.multi_hand_landmarks:
                image_height, image_width, _ = image.shape
                #annotated_image = image.copy()
                for hand_landmarks in results.multi_hand_landmarks:
                    print('hand_landmarks:', hand_landmarks)
                    print(f'Index finger tip coordinates: (',
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})')
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            cv2.imwrite("./test.jpg", cv2.flip(image, 1))
            cv2.imshow('MediaPipe Hands', image)

    else:
        cv2.imwrite("./test.jpg",decimg)
        cv2.imshow('SERVER',image) # 显示图像

    end = time.time()
    seconds = end - start
    fps  = 1/seconds
    conn.send(bytes(str(int(fps)),encoding='utf-8'))


In [ ]:
if __name__ == '__main__':

    address = ('192.168.1.14', 8001)
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    Detection_Flag=1

    Init(s)
    conn=Connect(s)

    while True:
        ReceiveVideo(conn)

        if cv2.waitKey(10) == 27: 
            break
            s.close()